In [0]:
!wget https://gist.githubusercontent.com/phillipj/4944029/raw/75ba2243dd5ec2875f629bf5d79f6c1e4b5a8b46/alice_in_wonderland.txt > /dev/null 2>&1
!mv alice_in_wonderland.txt wonderland.txt > /dev/null 2>&1

In [0]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.preprocessing.sequence import pad_sequences
from keras.optimizers import Adam
import re

In [0]:
# load ascii text and covert to lowercase
filename = "wonderland.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

# Split text at full stop
sentence_list = raw_text.split('\n')

raw_text = re.sub(r'[^\w\s]','',raw_text)
raw_text = ' '.join(raw_text.split())
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

# Remove all punctuations
for i in range(0, len(sentence_list)):
    sentence_list[i] = re.sub(r'[^\w\s]','',sentence_list[i])
    sentence_list[i] = ' '.join(sentence_list[i].split())
    
# Empty array to store int form of sentence ints
sentence_ints = [0] * len(sentence_list)
for i in range(0, len(sentence_list)):
    sentence_ints[i] = [0] * len(sentence_list[i])
    
for i in range(0, len(sentence_list)):
    for j in range(0, len(sentence_list[i])):
        sentence_ints[i][j] = char_to_int[sentence_list[i][j]]

In [0]:
char_to_int

{' ': 0,
 '0': 1,
 '3': 2,
 '_': 3,
 'a': 4,
 'b': 5,
 'c': 6,
 'd': 7,
 'e': 8,
 'f': 9,
 'g': 10,
 'h': 11,
 'i': 12,
 'j': 13,
 'k': 14,
 'l': 15,
 'm': 16,
 'n': 17,
 'o': 18,
 'p': 19,
 'q': 20,
 'r': 21,
 's': 22,
 't': 23,
 'u': 24,
 'v': 25,
 'w': 26,
 'x': 27,
 'y': 28,
 'z': 29}

In [0]:
import numpy as np
padded = pad_sequences(sequences=sentence_ints, maxlen=35, dtype='int', padding='post', value=0)
# padded = np.array(sentence_ints)

In [0]:
sentence_ints = [x for x in sentence_ints if x != []]

In [133]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  134159
Total Vocab:  30


In [0]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 35
dataX = padded
dataX = dataX[:-1]
dataY = []
for i in range(0, len(padded) - 1):
# 	seq_in = raw_text[i:i + seq_length]
# 	seq_out = raw_text[i + seq_length]
#   dataX.append([char_to_int[char] for char in seq_in])
    print(i)
    dataY.append(padded[i+1][0])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

In [0]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [0]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

# # define the LSTM model
# model = Sequential()
# # model.add(Dropout(0.1))
# model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=False))
# model.add(Dropout(0.2))
# #model.add(LSTM(256))
# model.add(Dense(y.shape[1], activation='softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='adam')


model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
adamOpt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
model.compile(loss='categorical_crossentropy', optimizer=adamOpt)

In [139]:
model.fit(X, y, epochs=100, batch_size=32, callbacks=callbacks_list)

Epoch 1/100
2716/2716 [==============================] - 21s 8ms/step - loss: 2.9740

Epoch 00001: loss improved from inf to 2.97404, saving model to weights-improvement-01-2.9740.hdf5
Epoch 2/100
2716/2716 [==============================] - 14s 5ms/step - loss: 2.9136

Epoch 00002: loss improved from 2.97404 to 2.91359, saving model to weights-improvement-02-2.9136.hdf5
Epoch 3/100
2716/2716 [==============================] - 14s 5ms/step - loss: 2.8995

Epoch 00003: loss improved from 2.91359 to 2.89950, saving model to weights-improvement-03-2.8995.hdf5
Epoch 4/100
2716/2716 [==============================] - 14s 5ms/step - loss: 2.9022

Epoch 00004: loss did not improve from 2.89950
Epoch 5/100
2716/2716 [==============================] - 14s 5ms/step - loss: 2.9024

Epoch 00005: loss did not improve from 2.89950
Epoch 6/100
2716/2716 [==============================] - 14s 5ms/step - loss: 2.8994

Epoch 00006: loss improved from 2.89950 to 2.89940, saving model to weights-improveme

KeyboardInterrupt: ignored

In [141]:
import sys
filename = "weights-improvement-56-2.3526.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = list(dataX[start])
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(500):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
"  ladderwhy i hadnt to bring but one "
     ttttt      ttttt      tttt      tttt      ttttt      tttttt      ttttt      tttt      tttt      ttttt      tttttt      ttttt      tttt      tttt      ttttt      tttttt      ttttt      tttt      tttt      ttttt      tttttt      ttttt      tttt      tttt      ttttt      tttttt      ttttt      tttt      tttt      ttttt      tttttt      ttttt      tttt      tttt      ttttt      tttttt      ttttt      tttt      tttt      ttttt      tttttt      ttttt      tttt      tttt      ttttt      tttttt    
Done.
